# Matrix Multiplication

## Dense matrix multiplication

In [3]:
import numpy as np

# message passing (MP)
x = np.array([2, 0])
A = np.array([[0, 2], [2, 0]])

y = x @ A

print(y)

[0 4]


In [3]:
# predicate activation
p = np.array([[[0.5, 0, 1]]])
A = np.array([[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 0], [0, 0]]])

_A = p.T*A
print(_A)

_A = _A.sum(0)
print(_A)

[[[0.  0.5]
  [0.5 0. ]]

 [[0.  0. ]
  [0.  0. ]]

 [[0.  0. ]
  [0.  0. ]]]
[[0.  0.5]
 [0.5 0. ]]


In [4]:
# predicate activation + MP
x = np.array([0.9, 0.1])
p = np.array([[[0.9, 0, 1]]])
A = np.array([[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 1], [0, 0]]])

_A = (p.T*A).sum(0)
print(_A)
y = x @ _A
print(y)

[[0.  1.9]
 [0.9 0. ]]
[0.09 1.71]


In [3]:
# score aggregation E1 E2 P1 P2

n_entities = 2
n_predicates = 3

A = np.array([[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 1], [0, 0]]])
x = np.array([[0, 0.5], [0.8, 0]])

p1 = np.array([[[0.9, 0, 1]]])
p2 = np.array([[[0, 1, 0]]])

ps = [p1, p2]

ye = np.zeros((x.shape[0], n_entities))
yp = np.zeros((len(ps), n_entities))
for i, p in enumerate(ps):
    _A = (p.T*A).sum(0)
    _y = x @ _A
    yp[i] = _y.sum(0)
    ye += _y
    print(_y)

y = np.vstack([ye,yp])
print(y)
print(y.sum(0))

# activations across components
threshold = 0
a = np.where(y>threshold)[1]
np.bincount(a)

[[0.45 0.  ]
 [0.   1.52]]
[[0.5 0. ]
 [0.  0. ]]
[[0.95 0.  ]
 [0.   1.52]
 [0.45 1.52]
 [0.5  0.  ]]
[1.9  3.04]


array([3, 2])

## Sparse matrix multiplication

In [2]:
import scipy.sparse as sp

# message passing (MP)
x = sp.csr_matrix(np.array([2, 0]))
A = sp.csr_matrix(np.array([[0, 2], [2, 0]]))

y = x @ A

print(y)

NameError: name 'np' is not defined

In [8]:
# predicate activation
p = np.array([0.5, 0, 1])
A = [[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 0], [0, 0]]]
A = [sp.csr_matrix(a) for a in A]
# print(A)

_A = p*A
print(_A)

_A = sum(_A)
print(_A)

[<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>
 <2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>
 <2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>]
  (0, 1)	0.5
  (1, 0)	0.5


In [9]:
# predicate activation + MP
x = sp.csr_matrix(np.array([0.9, 0.1]))
p = np.array([0.9, 0, 1])
A = [[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 1], [0, 0]]]
A = [sp.csr_matrix(a) for a in A]

_A = sum(p*A)
print(_A)

y = x @ _A
print(y)

  (0, 1)	1.9
  (1, 0)	0.9
  (0, 0)	0.09000000000000001
  (0, 1)	1.71


In [22]:
# score aggregation E1 E2 P1 P2
from sklearn.preprocessing import normalize, binarize

threshold = 0
n_entities = 2

A = [[[0, 1], [1, 0]], [[0, 0], [1, 0]], [[0, 1], [0, 0]]]
A = [sp.csr_matrix(a) for a in A]
x = sp.csr_matrix(np.array([[0.9, 0.5], [0.8, 0.1]]))

p1 = np.array([0.9, 0, 1])
p2 = np.array([0, 1, 0])

ps = [p1, p2]

ye = sp.csr_matrix((x.shape[0], n_entities))
yp = sp.csr_matrix((len(ps), n_entities))
for i, p in enumerate(ps):
    _A = sum(p*A)
    _y = x @ _A
    yp[i] = _y.sum(0)
    ye += _y

y = sp.vstack([ye,yp])
print(y)
sum_a = sum(y)
sum_a_norm = normalize(sum_a, norm='max', axis=1).toarray()[0]
print(sum_a_norm)

# activations across components
y_counts = binarize(y, threshold=0.0)
print(w_normalized)
count_a = sum(y_counts).toarray()[0]
print(count_a)

scores = sum_a_norm + count_a
print(scores)

  (0, 0)	0.95
  (0, 1)	1.71
  (1, 0)	0.19
  (1, 1)	1.52
  (2, 0)	0.54
  (2, 1)	3.23
  (3, 0)	0.6
  (3, 1)	0.0
[0.35294118 1.        ]
  (0, 0)	1.0
  (0, 1)	1.0
  (1, 0)	1.0
  (1, 1)	1.0
  (2, 0)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
[4. 3.]
[4.35294118 4.        ]


/home/zola/anaconda3/envs/tf36/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
